In [152]:
from openai import OpenAI
from datetime import datetime
import json

In [143]:
client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = "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"
)

In [153]:
client_ = OpenAI(
    base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1",
    api_key="sk-bd4b49009cf749c5b795950f77314c1d"
)

In [154]:
# Define a function that can be called by the AI
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    # This is a mock function; in real-world scenarios, you'd fetch this data from a weather API
    weather_data = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "description": "sunny"
    }
    return json.dumps(weather_data)

def get_current_time():
    # Obtain the current date and time
    current_datetime = datetime.now()
    # Format the current date and time
    formatted_time = current_datetime.strftime('%Y-%m-%d %H:%M:%S')
    # Return the formatted current date and time
    return f"Current time: {formatted_time}."

# Function description for OpenAI to understand when to use this function
tools = [
    # Tool 1: obtain the current time
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "This tool can help you query the current time.",
            # No request parameter is needed. The parameters parameter is left empty
            "parameters": {}
        }
    },  
    # Tool 2: obtain the weather of a specific city
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "This tool can help you query the weather of a city.",
            "parameters": {  
                "type": "object",
                "properties": {
                    # The parameters parameter is set to location, which specifies the location whose weather you want to query
                    "location": {
                        "type": "string",
                        "description": "A city, county, or district, such as Beijing, Hangzhou, or Yuhang."
                    }
                }
            },
            "required": [
                "location"
            ]
        }
    }
]

In [145]:
def run_conversation():
    messages = [{"role": "user", "content": "What’s the weather like in Boston today?"}]
    
    response = client.chat.completions.create(
        model="meta/llama-3.3-70b-instruct",  # Use a model that supports function calling
        messages=messages,
        tools=[{"type":"function","function":{"name":"get_current_weather","description":"Returns the current weather at a location, if one is specified, and defaults to the user's location.","parameters":{"type":"object","properties":{"location":{"type":"string","description":"The location to find the weather of, or if not provided, it's the default location."},"format":{"type":"string","enum":["celsius","fahrenheit"],"description":"Whether to use SI or USCS units (celsius or fahrenheit)."}},"required":[]}}}],
        tool_choice="auto",  # Auto means let the model decide when to call a function
        stream=True  # Enable streaming
    )

    function_call_in_progress = False
    function_args = ''
    for chunk in response:
        delta = chunk.choices[0].delta
        print(delta)


        if 'content' in delta and delta['content']:
            print(delta['content'], end='', flush=True)


        function_results = []
             
        # Handle tool calls
        if delta.tool_calls is not None:
            print('tool calling...')
            for tool_call in delta.tool_calls:
                function_name = tool_call.function.name
                print(f"\nCalling function: {function_name}")
                function_call_in_progress = True
                function_args = tool_call.function.arguments
                function_args_json = json.loads(function_args)

                if function_call_in_progress:
                    if function_name == "get_current_weather":
                        func_response = get_current_weather(
                            location=function_args_json.get("location"),
                            unit=function_args_json.get("format", "fahrenheit")
                        )
                        function_results.append(
                            {
                                "fn_name": function_name,
                                "fn_args": function_args_json,
                                "fn_response": func_response
                            }
                        )
                    function_call_in_progress = False 
                    function_args = ""

    messages.append({
        "role": "tool",
        "content": function_results[0]['fn_response'],
        "name": function_results[0]['fn_name']
    })
    
    response = client.chat.completions.create(
        model="meta/llama-3.3-70b-instruct",  # Use a model that supports function calling
        messages=messages,
        stream=True  # Enable streaming
    )
    for chunk in response:
        print(chunk.choices[0].delta.content)


In [155]:
def run_conversation():
    messages = [{"role": "user", "content": "What’s the weather like in Boston today?"}]
    
    response = client_.chat.completions.create(
        model="qwen-max",  # Use a model that supports function calling
        messages=messages,
        tools=tools,
        tool_choice="auto",  # Auto means let the model decide when to call a function
        stream=True  # Enable streaming
    )

    function_call_in_progress = False
    function_args = ''
    for chunk in response:
        delta = chunk.choices[0].delta
        print(delta)


        if 'content' in delta and delta['content']:
            print(delta['content'], end='', flush=True)


        function_results = []
             
        # Handle tool calls
        if delta.tool_calls is not None:
            print('tool calling...')
            for tool_call in delta.tool_calls:
                function_name = tool_call.function.name
                print(f"\nCalling function: {function_name}")
                function_call_in_progress = True
                function_args = tool_call.function.arguments
                function_args_json = json.loads(function_args)

                if function_call_in_progress:
                    if function_name == "get_current_weather":
                        func_response = get_current_weather(
                            location=function_args_json.get("location"),
                            unit=function_args_json.get("format", "fahrenheit")
                        )
                        function_results.append(
                            {
                                "fn_name": function_name,
                                "fn_args": function_args_json,
                                "fn_response": func_response
                            }
                        )
                    function_call_in_progress = False 
                    function_args = ""

    messages.append({
        "role": "tool",
        "content": function_results[0]['fn_response'],
        "name": function_results[0]['fn_name']
    })
    
    response = client.chat.completions.create(
        model="meta/llama-3.3-70b-instruct",  # Use a model that supports function calling
        messages=messages,
        stream=True  # Enable streaming
    )
    for chunk in response:
        print(chunk.choices[0].delta.content)

In [156]:
run_conversation()

ChoiceDelta(content=None, function_call=None, refusal=None, role='assistant', tool_calls=[ChoiceDeltaToolCall(index=0, id='call_f5dd12422a8249129bdc50', function=ChoiceDeltaToolCallFunction(arguments='{"location": "', name='get_current_weather'), type='function')])
tool calling...

Calling function: get_current_weather


JSONDecodeError: Unterminated string starting at: line 1 column 14 (char 13)